In [15]:
import cv2
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import os
import tempfile
from cheada.cloud_service_agent.s3 import s3_utils


In [31]:
def convert_images_to_pdf(image_folder, output_pdf):
    # 페이지 사이즈 정의 (A4)
    page_width, page_height = A4

    # 이미지 파일 목록 가져오기
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

    # 이미지 파일 크기 나누기
    threshold = 500
    large_problems = []
    small_problems = []
    img_path_list = []
    for file in image_files:
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        img_height, img_width, _ = img.shape
        if img_height >= threshold:
            large_problems.append(img)
        else:
            small_problems.append(img)
        img_path_list.append(img_path)

    c = canvas.Canvas(output_pdf, pagesize=A4)
    c.drawImage("template.png", 0, 0, width=page_width, height=page_height)

    # Large images processing (remains unchanged)
    left_page_full = False
    for i, img in enumerate(large_problems):
        img_height, img_width, _ = img.shape
        if left_page_full:
            c.drawImage(img_path_list[i], page_width / 2, page_height - img_height - 20)
            c.showPage()
        else:
            c.drawImage(img_path_list[i], 10, page_height - img_height - 20)
        left_page_full = not left_page_full  # 수정된 부분

    # Small images processing
    for i, img in enumerate(small_problems):
        img_height, img_width, _ = img.shape
        # 이미지 배치 좌표 설정
        positions = [
            (21, page_height - img_height - 21),  # 1
            (316, page_height - img_height - 21),  # 2
            (21, page_height - img_height - 427),  # 3
            (316, page_height - img_height - 427)  # 4
        ]
        
        pos = positions[i % 4]  # 이미지의 위치 선택
        print(f"index {i} : {pos}")

        c.drawImage(img_path_list[i], pos[0], pos[1])

        if i % 4 == 3:
            c.showPage()

    c.save()
    print(f"PDF 파일이 성공적으로 생성되었습니다: {output_pdf}")

def save_temp_image(image):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_path = temp_file.name
    cv2.imwrite(temp_path, image)
    return temp_path

In [32]:
# 이미지 폴더 경로 및 출력 PDF 파일 이름 설정
image_folder = os.path.abspath("temp_problem_storage")

textbook_name = "블랙라벨 수학 II2"
output_pdf = f'{textbook_name}_editted.pdf'

convert_images_to_pdf('/Users/mac/PycharmProjects/chaeda_fastapi/cheada/domain/review_note_maker/service/temp_problem_storage', output_pdf)

index 0 : (21, 698.8897637795277)
index 1 : (316, 668.8897637795277)
index 2 : (21, 275.8897637795277)
index 3 : (316, 260.8897637795277)
PDF 파일이 성공적으로 생성되었습니다: 블랙라벨 수학 II2_editted.pdf


In [ ]:
def download_textbook_from_s3(filename, file_location):
    """
    
    :param filename: 
    :param file_location: 
    :return: 
    """
    s3_utils.download_file_from_s3(filename, file_location)